In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Carregando {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Carregando {file}')
        loader = Docx2txtLoader(file)
    else:
        print('Formato não suportado!')
        return None

    data = loader.load()
    return data

# Wikipedia Loader
def load_from_wikipedia(query, lang='pt', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data



In [ ]:
load_document('docs/clt.pdf')

In [ ]:
def chunk_data(data, chunk_size=1000):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

In [ ]:
data=load_document('docs/clt.pdf')
chunks=chunk_data(data)

In [ ]:
def embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_tokens / 1000 * 0.0001:.6f}')

In [ ]:
embedding_cost(chunks)


In [ ]:
def insert_embeddings(index_name):
    import pinecone
    from pinecone import PodSpec
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings
    environment = "gcp-starter"
    spec = PodSpec(environment=environment)

    embeddings = OpenAIEmbeddings()
    pc=pinecone.Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

    if index_name in pc.list_indexes():
        print(f'Index {index_name}')
        vector_store = pc.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Criando index {index_name}')
        pc.create_index(index_name, dimension=1536, metric='cosine', spec=spec)
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
    return vector_store

In [ ]:
def delete_index(index_name='all'):
    import pinecone
    pc=pinecone.Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

    if index_name == 'all':
        indexes = pc.list_indexes()
        print('Deletando todos os índices...')
        for index in indexes:
            pc.delete_index(index)
    else:
        print(f'Deletando índice {index_name}...')
        pc.delete_index(index_name)

In [ ]:
index_name = 'linuxtips'
delete_index(index_name=index_name)

In [ ]:
index_name = 'linuxtips'
vector_store = insert_embeddings(index_name)

In [ ]:
def get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(q)
    return answer

def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history

In [ ]:
q = 'em que ano a clt foi criada?'
answer = get_answer(vector_store, q)
print(answer)

In [ ]:
import time
i = 1
print('Digite sair para encerrar.')
while True:
    q = input(f'Pergunta: #{i}: ')
    i = i+1
    if q.lower() in ['sair']:
        print('Encerrando...')
        time.sleep(2)
        break

    answer = get_answer(vector_store, q)
    print(f'\nResposta: {answer}')
    print(f'\n {"-" * 50} \n')

In [ ]:
chat_history = []
question = 'em que ano a clt foi criada?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])

In [ ]:
question = 'estamos em 2023, quantos anos passaram?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])

In [ ]:
print(chat_history)